# Kim & Zhou et al., Nature, 2022 (gene expression)

Author: Julian Q. Zhou

Docker container: `julianqz/wu_cimm:ref_0.1.1_lsf` (Python 3.8.8)

Memory intensive. Recommend allocating at least 120GB memory.

The code below was run on a HPC environment that did not support running Jupyter Lab via a browser.

As such, key console outputs were pasted as comments. Visualizations were outputted as pdfs or pngs.

A tsv file was exported at the end for overlaying S-binding specificity on the UMAPs in the R script.

## Load packages & config

In [ ]:
from pathlib import Path
import os
import copy
import re
import math
import numpy as np
import pandas as pd
import scanpy as sc
from anndata import read_h5ad
from anndata import AnnData
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# scanpy settings

# verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.verbosity = 3             
sc.logging.print_header()

In [ ]:
# scanpy==1.7.2 anndata==0.7.6 umap==0.5.1 numpy==1.20.2 scipy==1.6.2 pandas==1.2.3 scikit-learn==0.24.1 statsmodels==0.12.2 python-igraph==0.9.4

In [ ]:
sc.settings.set_figure_params(dpi=120, dpi_save=150, vector_friendly=False, format="pdf", 
                              transparent=False, facecolor="w", color_map="viridis_r")

In [ ]:
sc.settings.figdir = "."

In [ ]:
# resolution for clustering all cells
res_1 = 0.23
anno_col_1 = f"anno_leiden_{res_1:.2f}"

In [ ]:
# resolution for clustering B cells
res_2 = 0.18
anno_col_2 = f"anno_leiden_{res_2:.2f}"

In [ ]:
# number of principal components to use
N_PC = 20

In [ ]:
# type of embedding
eb = "umap"

In [ ]:
# filenames
fn_1 = "WU368_kim_et_al_nature_2022_gex_all_cells.h5ad"
fn_2 = "WU368_kim_et_al_nature_2022_gex_b_cells.h5ad"
fn_save = f"gex_b_cells_{eb}_anno.tsv.gz"

## All cells

### AnnData containing clustering results of all cells

In [ ]:
adata_1 = read_h5ad(fn_1)

In [ ]:
# reset .X (previously set to `None` in order to reduce file size)
adata_1.X = adata_1.layers["log_norm"]

In [ ]:
# check that column containing annotation labels is present
assert anno_col_1 in adata_1.obs.keys()

In [ ]:
# exclude cluster annotated as "B & T"
clusters_excl_1 = ["B & T"]
bool_incl_1 = ~adata_1.obs[anno_col_1].isin(clusters_excl_1)

# subset
adata_1 = adata_1[bool_incl_1, :]

# remove excluded categories
adata_1.obs[anno_col_1] = adata_1.obs[anno_col_1].cat.remove_unused_categories()

# re-run dendrogram
sc.tl.dendrogram(adata_1, groupby=anno_col_1, n_pcs=N_PC, use_rep="X_pca")

In [ ]:
adata_1.shape

In [ ]:
# (383608, 15842)

### Cell counts by annotation

In [ ]:
adata_1.obs[anno_col_1].value_counts()

In [ ]:
# B           193442
# CD4+ T      136978
# CD8+ T       36613
# NK            6986
# Monocyte      6520
# pDC           2425
# FDC            644
# Name: anno_leiden_0.23, dtype: int64

In [ ]:
# stratified by tissue
pd.crosstab(index=adata_1.obs["tissue"], columns=adata_1.obs[anno_col_1], margins=True)

In [ ]:
# anno_leiden_0.23       B  CD4+ T  CD8+ T  FDC  Monocyte    NK   pDC     All
# tissue                                                                     
# LN                166022  136929   36532  644      6379  6268  2420  355194
# blood              27420      49      81    0       141   718     5   28414
# All               193442  136978   36613  644      6520  6986  2425  383608

### Dot plot (Extended Data Fig 2c)

In [ ]:
genes_dict_1 = {
    "B": ["MS4A1", "CD19", "CD79A"],
    "T": ["CD3D", "CD3E", "CD3G", "IL7R"],
    "CD4+ T": ["CD4"],
    "CD8+ T": ["CD8A"],
    "NK": ["GZMB", "GNLY", "NKG7", "NCAM1"],
    "Monocyte": ["CD14", "LYZ", "CST3", "MS4A7"],
    "pDC": ["IL3RA", "CLEC4C"],
    "FDC": ["FDCSP", "CXCL14", "FCAMR"]
}

genes_lst_1 = [x for v in genes_dict_1.values() for x in v]

In [ ]:
# check that all genes for visualization are present in count matrix
assert all( [x in adata_1.var["gene_name"] for x in genes_lst_1] )

In [ ]:
anno_order_1 = ["B", "CD4+ T", "CD8+ T", "NK", "Monocyte", "pDC", "FDC"]

In [ ]:
cur_fig = sc.pl.dotplot(adata_1, layer="log_norm", var_names=genes_dict_1, groupby=anno_col_1,
                        dendrogram=False, 
                        categories_order=anno_order_1, swap_axes=False,
                        cmap="Blues", return_fig=True, save=False)
cur_fig.savefig("gex_all_cells_dot.pdf", bbox_inches="tight")
plt.close()

### UMAPs (Fig 2a; Extended Data Fig 2b)

In [ ]:
# color palette
anno_palette_1 = {
    "B": "violet", 
    "CD4+ T": "dodgerblue",
    "CD8+ T": "orange",
    "NK": "purple", 
    "Monocyte": "seagreen", 
    "pDC": "darkgray",
    "FDC": "red"
}

In [ ]:
# combined (Extended Data Fig 2b, left)
cur_fig = sc.pl.embedding(adata_1, basis=f"X_{eb}", color=anno_col_1, 
                          size=3, palette=anno_palette_1, 
                          legend_loc="right", legend_fontsize=0, legend_fontoutline=0,
                          frameon=True, ncols=1, title="",
                          return_fig=True, save=False)

cur_fig.savefig(f"gex_all_cells_{eb}_combined.png", dpi=500, bbox_inches="tight")

plt.close(cur_fig)

In [ ]:
# DataFrame containing UMAP coordinates and select metadata columns
eb_df_anno_1 = pd.concat(
    [ pd.DataFrame(adata_1.obsm[f"X_{eb}"], columns=[f"{eb}_x", f"{eb}_y"], index=adata_1.obs.index),
      adata_1.obs.loc[:, [anno_col_1, "tissue", "donor"]] ], 
    axis=1)

In [ ]:
# remove grid lines and ticks
sns.set_style("white")

In [ ]:
# stratified by tissue (Fig 2a, left)

# a Facet.Grid
cur_fig = sns.relplot(x=f"{eb}_x", y=f"{eb}_y", data=eb_df_anno_1, 
                      hue=anno_col_1, palette=anno_palette_1,
                      hue_order=anno_order_1, # legend order
                      col="tissue", 
                      s=3, alpha=0.85, 
                      legend=True,
                      height=5, aspect=0.9)

# remove x-axis label
cur_fig.set(xlabel=None)
# remove y-axis label
cur_fig.set(ylabel=None)
# remove tick labels
cur_fig.set(xticklabels=[])
cur_fig.set(yticklabels=[])

# set legend title
cur_fig._legend.set_title("")

cur_fig.savefig(f"gex_all_cells_{eb}_by_tissue.png", dpi=500, bbox_inches="tight")

plt.close()

In [ ]:
# stratified by donor (Extended Data Fig 2b, right)

max_per_row = 4

# a Facet.Grid
cur_fig = sns.relplot(x=f"{eb}_x", y=f"{eb}_y", data=eb_df_anno_1, 
                      hue=anno_col_1, palette=anno_palette_1,
                      hue_order=anno_order_1, # legend order
                      col="donor", col_wrap=max_per_row,
                      s=3, alpha=0.85, 
                      height=5, aspect=0.9)

# remove x-axis label
cur_fig.set(xlabel=None)
# remove y-axis label
cur_fig.set(ylabel=None)
# remove tick labels
cur_fig.set(xticklabels=[])
cur_fig.set(yticklabels=[])

# set legend title
cur_fig._legend.set_title("")


cur_fig.savefig(f"gex_all_cells_{eb}_by_donor.png", dpi=120, bbox_inches="tight")

plt.close()

In [ ]:
del adata_1, eb_df_anno_1

## B cells

Tip: When running the `B cells` section after running the `All cells` section, if memory limit is used, skip running `All cells` and go straight to `B cells`. These two sections should be independent of each other.

### AnnData containing clustering results of B cells

In [ ]:
adata_2 = read_h5ad(fn_2)

In [ ]:
# reset .X (previously set to `None` in order to reduce file size)
adata_2.X = adata_2.layers["log_norm"]

In [ ]:
# check that column containing annotation labels is present
assert anno_col_2 in adata_2.obs.keys()

In [ ]:
# exclude cluster annotated as "B & T"
clusters_excl_2 = ["B & T"]
bool_incl_2 = ~adata_2.obs[anno_col_2].isin(clusters_excl_2)

# subset
adata_2 = adata_2[bool_incl_2, :]

# remove excluded categories
adata_2.obs[anno_col_2] = adata_2.obs[anno_col_2].cat.remove_unused_categories()

# re-run dendrogram
sc.tl.dendrogram(adata_2, groupby=anno_col_2, n_pcs=N_PC, use_rep="X_pca")

In [ ]:
adata_2.shape

In [ ]:
# (182645, 15842)

### Cell counts by annotation

In [ ]:
adata_2.obs[anno_col_2].value_counts()

In [ ]:
# GC         62156
# RMB        42255
# Naive      38686
# PB         27231
# LNPC       12299
# PB-like       18
# Name: anno_leiden_0.18, dtype: int64

In [ ]:
# stratified by tissue
pd.crosstab(index=adata_2.obs["tissue"], columns=adata_2.obs[anno_col_2], margins=True)

In [ ]:
# anno_leiden_0.18     GC   LNPC  Naive     PB    RMB  PB-like     All
# tissue                                                              
# LN                62156  12299  38665      0  42105        0  155225
# blood                 0      0     21  27231    150       18   27420
# All               62156  12299  38686  27231  42255       18  182645

### Dot plot (Extended Data Fig 2e)

In [ ]:
genes_dict_2 = {
    "GC": ["BCL6", "RGS13", "MEF2B", "STMN1", "ELL3", "SERPINA9"],
    "PB/LNPC": ["XBP1", "IRF4", "SEC11C", "FKBP11", "JCHAIN", "PRDM1"],
    "Naive": ["TCL1A", "IL4R", "CCR7", "IGHM", "IGHD"],
    "RMB": ["TNFRSF13B", "CD27", "CD24"]
}

genes_lst_2 = [x for v in genes_dict_2.values() for x in v]

In [ ]:
# check that all genes for visualization are present in count matrix
assert all( [x in adata_2.var["gene_name"] for x in genes_lst_2] )

In [ ]:
anno_order_2 = ["Naive", "PB", "PB-like", "GC", "LNPC", "RMB"]

In [ ]:
cur_fig = sc.pl.dotplot(adata_2, layer="log_norm", var_names=genes_dict_2, groupby=anno_col_2,
                        dendrogram=False, 
                        categories_order=anno_order_2, swap_axes=False,
                        cmap="Blues", return_fig=True, save=False)
cur_fig.savefig("gex_b_cells_dot.pdf", bbox_inches="tight")
plt.close()

### UMAPs (Fig 2a; Extended Data Fig 2d)

In [ ]:
# color palette
anno_palette_2 = {
    "GC": "dodgerblue", 
    "LNPC": "forestgreen",
    "PB": "red",
    "Naive": "orange",
    "RMB": "violet",
    "PB-like": "brown"
}

In [ ]:
# combined (Extended Data Fig 2d, left)
cur_fig = sc.pl.embedding(adata_2, basis=f"X_{eb}", color=anno_col_2, 
                          size=3, palette=anno_palette_2, 
                          legend_loc="right", legend_fontsize=0, legend_fontoutline=0,
                          frameon=True, ncols=1, title="",
                          return_fig=True, save=False)

cur_fig.savefig(f"gex_b_cells_{eb}_combined.png", dpi=500, bbox_inches="tight")

plt.close(cur_fig)

In [ ]:
# DataFrame containing UMAP coordinates and select metadata columns
eb_df_anno_2 = pd.concat(
    [ pd.DataFrame(adata_2.obsm[f"X_{eb}"], columns=[f"{eb}_x", f"{eb}_y"], index=adata_2.obs.index),
      adata_2.obs.loc[:, [anno_col_2, "tissue", "donor", "cell_id"]] ], 
    axis=1)

In [ ]:
# remove grid lines and ticks
sns.set_style("white")

In [ ]:
# stratified by tissue (Fig 2a, right)

# a Facet.Grid
cur_fig = sns.relplot(x=f"{eb}_x", y=f"{eb}_y", data=eb_df_anno_2, 
                      hue=anno_col_2, palette=anno_palette_2,
                      hue_order=anno_order_2, # legend order
                      col="tissue", 
                      s=3, alpha=0.85, 
                      legend=True,
                      height=5, aspect=0.9)

# remove x-axis label
cur_fig.set(xlabel=None)
# remove y-axis label
cur_fig.set(ylabel=None)
# remove tick labels
cur_fig.set(xticklabels=[])
cur_fig.set(yticklabels=[])

# set legend title
cur_fig._legend.set_title("")

cur_fig.savefig(f"gex_b_cells_{eb}_by_tissue.png", dpi=500, bbox_inches="tight")

plt.close()

In [ ]:
# stratified by donor (Extended Data Fig 2d, right)

max_per_row = 4

# a Facet.Grid
cur_fig = sns.relplot(x=f"{eb}_x", y=f"{eb}_y", data=eb_df_anno_2, 
                      hue=anno_col_2, palette=anno_palette_2,
                      hue_order=anno_order_2, # legend order
                      col="donor", col_wrap=max_per_row,
                      s=3, alpha=0.85, 
                      height=5, aspect=0.9)

# remove x-axis label
cur_fig.set(xlabel=None)
# remove y-axis label
cur_fig.set(ylabel=None)
# remove tick labels
cur_fig.set(xticklabels=[])
cur_fig.set(yticklabels=[])

# set legend title
cur_fig._legend.set_title("")


cur_fig.savefig(f"gex_b_cells_{eb}_by_donor.png", dpi=120, bbox_inches="tight")

plt.close()

### Export DataFrame

In [ ]:
# used in R script for Extended Data Fig 2h
eb_df_anno_2.to_csv(fn_save, sep="\t", header=True, index=True, compression="gzip")